# Create Dataset from .zip files and calc normalization statistics

Step_1: create standard dataset to be re-used, either by creating folders or as hdf5 object. 

Step_2: calculate mean + std for train and test images

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
from common import shuffle_zip
BASE = "datasets/yeast_Actin_v1/"

## Run scripts to generate ready_data

In [4]:
ds_name = BASE
shuffle_zip(ds_name+"/original_zips",ds_name+'/shuffled_zips/',ds_name)

working on [01_mfb1KO] class
deleted 49 non square images.
{True}
296 0
length of train: [226]; length of test: [70]
Files moved to:datasets/yeast_Actin_v1//shuffled_zips/
train created
train/01_mfb1KO created
test created
test/01_mfb1KO created
working on [01_mmm1KO] class
deleted 37 non square images.
{True}
237 0
length of train: [167]; length of test: [70]
Files moved to:datasets/yeast_Actin_v1//shuffled_zips/
working on [01_mmr1KO] class
deleted 12 non square images.
{True}
249 0
length of train: [179]; length of test: [70]
Files moved to:datasets/yeast_Actin_v1//shuffled_zips/
working on [01_num1KO] class
deleted 7 non square images.
{True}
114 0
length of train: [44]; length of test: [70]
Files moved to:datasets/yeast_Actin_v1//shuffled_zips/
working on [01_WT] class
deleted 22 non square images.
{True}
188 0
length of train: [118]; length of test: [70]
Files moved to:datasets/yeast_Actin_v1//shuffled_zips/
working on [02_mfb1KO] class
deleted 39 non square images.
{False}
0 251